In [34]:
# --- INSTALLATIONS ET IMPORTS ---
try:
    import gdown
except ImportError:
    import os
    os.system("pip install -q gdown")
    import gdown

import os
import glob

import numpy as np
import pandas as pd
from scipy import stats

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader

In [35]:
# ==========================================
# 1. TÉLÉCHARGEMENT ET INSPECTION
# ==========================================

dossier_nom = "redshift"
url = "https://drive.google.com/drive/folders/1-tQH6rfB1XoF7ml98yqVn7Z2IQwGMUdK?usp=sharing"

if os.path.exists(dossier_nom):
    print(f"✅ Le dossier '{dossier_nom}' est déjà présent. Pas besoin de retélécharger !")
else:
    print(f"📥 Dossier introuvable. Téléchargement en cours...")
    gdown.download_folder(url, quiet=False, use_cookies=False)
    print("✅ Téléchargement terminé !")

pattern = os.path.join(dossier_nom, "*")
dossier_local = glob.glob(pattern)

print(f"\nSuccès ! {len(dossier_local)} fichiers trouvés dans '{dossier_nom}'.")
print("Détail des fichiers :")

for chemin in dossier_local:
    dossier = os.path.dirname(chemin)
    fichier = os.path.basename(chemin)
    print(f"[{dossier}] :\t {fichier}")

print("\n--- INSPECTION APPROFONDIE DE 'redshift' ---")
dossier_cible = "redshift"
est_vide = True

for root, dirs, files in os.walk(dossier_cible):
    for name in files:
        est_vide = False
        chemin_complet = os.path.join(root, name)
        taille = os.path.getsize(chemin_complet)
        print(f"📄 TROUVÉ : {chemin_complet} | Taille : {taille} octets")

if est_vide:
    print("⚠️ LE DOSSIER EST VIDE.")
    print("Cela veut dire que Google a bloqué le téléchargement du contenu.")
    print("Solution : Vérifie que les fichiers DANS le dossier Drive sont aussi en 'Public'.")

✅ Le dossier 'redshift' est déjà présent. Pas besoin de retélécharger !

Succès ! 17 fichiers trouvés dans 'redshift'.
Détail des fichiers :
[redshift] :	 XMM_LSS_v11_uijk_0177_spec_UD.npz
[redshift] :	 info2.csv
[redshift] :	 info3.csv
[redshift] :	 XMM_LSS_v11_uijk_0006_spec_D.npz
[redshift] :	 info4.csv
[redshift] :	 cnn_photoz_model.h5
[redshift] :	 tmp_csv
[redshift] :	 XMM_LSS_v11_uijk_0162_phot_UD.npz
[redshift] :	 output_zphot_only.txt
[redshift] :	 COSMOS_v11_uijk_0073_spec_UD.npz
[redshift] :	 XMM_LSS_v11_uijk_0162_phot_D.npz
[redshift] :	 cnn_architecture.png
[redshift] :	 COSMOS_v11_uijk_0020_spec_D.npz
[redshift] :	 COSMOS_v11_uijk_0001_photo_D.npz
[redshift] :	 best_photoz_model.pt
[redshift] :	 COSMOS_v11_uijk_0213_photo_UD.npz
[redshift] :	 info.csv

--- INSPECTION APPROFONDIE DE 'redshift' ---
📄 TROUVÉ : redshift/XMM_LSS_v11_uijk_0177_spec_UD.npz | Taille : 1923288 octets
📄 TROUVÉ : redshift/info2.csv | Taille : 4008 octets
📄 TROUVÉ : redshift/info3.csv | Taille : 5217

In [36]:
# ==========================================
# 2. RÉCUPÉRATION ET CHARGEMENT
# ==========================================

path_pattern = os.path.join("redshift", "*.npz")
fichiers_npz = glob.glob(path_pattern)
print(f"\n📂 Fichiers .npz trouvés ({len(fichiers_npz)}):")
for i, fichier in enumerate(fichiers_npz):
    print(f"  {i+1}. {fichier}")

cosmos_files = [f for f in fichiers_npz if "COSMOS" in f]
print(f"\nFichiers COSMOS ({len(cosmos_files)}):")
for i, fichier in enumerate(cosmos_files):
    print(f"  {i+1}. {fichier}")

data_list = []
for fichier in cosmos_files:
    try:
        data = np.load(fichier, allow_pickle=True)
        data_list.append(data)
        print(f"✅ Chargé: {fichier}")
    except Exception as e:
        print(f"❌ Erreur en chargeant {fichier}: {e}")

if not data_list:
    raise ValueError("Aucun fichier COSMOS chargé. Arrêt du script.")

print(f"Total de fichiers COSMOS chargés: {len(data_list)}")

for i, data in enumerate(data_list):
    print(f"\nClés dans le fichier {cosmos_files[i]}:")
    for key in data.keys():
        print(f"  - {key}")
        try:
            print(f"    Shape de {key}: {data[key].shape}")
        except:
            pass

bands_all = ["u","g","r","i","z","y","J","H"]
info = data_list[-1]['info'] # On prend l'info du dernier fichier chargé
print(f"\nFichiers dans la dernière archive : {data_list[-1].files}")
print(f"Noms des colonnes info : {info.dtype.names}")


📂 Fichiers .npz trouvés (8):
  1. redshift/XMM_LSS_v11_uijk_0177_spec_UD.npz
  2. redshift/XMM_LSS_v11_uijk_0006_spec_D.npz
  3. redshift/XMM_LSS_v11_uijk_0162_phot_UD.npz
  4. redshift/COSMOS_v11_uijk_0073_spec_UD.npz
  5. redshift/XMM_LSS_v11_uijk_0162_phot_D.npz
  6. redshift/COSMOS_v11_uijk_0020_spec_D.npz
  7. redshift/COSMOS_v11_uijk_0001_photo_D.npz
  8. redshift/COSMOS_v11_uijk_0213_photo_UD.npz

Fichiers COSMOS (4):
  1. redshift/COSMOS_v11_uijk_0073_spec_UD.npz
  2. redshift/COSMOS_v11_uijk_0020_spec_D.npz
  3. redshift/COSMOS_v11_uijk_0001_photo_D.npz
  4. redshift/COSMOS_v11_uijk_0213_photo_UD.npz
✅ Chargé: redshift/COSMOS_v11_uijk_0073_spec_UD.npz
✅ Chargé: redshift/COSMOS_v11_uijk_0020_spec_D.npz
✅ Chargé: redshift/COSMOS_v11_uijk_0001_photo_D.npz
✅ Chargé: redshift/COSMOS_v11_uijk_0213_photo_UD.npz
Total de fichiers COSMOS chargés: 4

Clés dans le fichier redshift/COSMOS_v11_uijk_0073_spec_UD.npz:
  - cube
    Shape de cube: (12, 64, 64, 9)
  - info
    Shape de info: (

In [37]:
# ==========================================
# 3. CONCATÉNATION ET PRÉPARATION DES DONNÉES
# ==========================================

print("\n--- 3. PRÉPARATION DES DONNÉES GLOBALES ---")

bands = ["u", "g", "r", "i", "z", "y", "J", "H"]

mags_list = []
zspec_list = []
zphot_list = []
cube_list = []

for d in data_list:
    info_array = d['info']
    
    mags = np.column_stack([info_array[b] for b in bands])
    mags_list.append(mags)
    zphot_list.append(info_array['ZPHOT'])
    cube_list.append(d['cube'])
    
    # Extraction de ZSPEC avec protection NaN
    try:
        zspec_list.append(info_array['ZSPEC'])
    except ValueError:
        print(f"⚠️ ZSPEC introuvable dans un fichier. Remplissage avec des NaN.")
        zspec_list.append(np.full(info_array.shape[0], np.nan))

all_mags = np.vstack(mags_list)
all_zphot = np.hstack(zphot_list)
all_zspec = np.hstack(zspec_list)
all_cubes = np.vstack(cube_list)

print(f"✅ Magnitudes extraites : {all_mags.shape}")
print(f"✅ ZSPEC (Vérité Terrain / Test) extraits : {all_zspec.shape}")
print(f"✅ ZPHOT (Estimation / Train) extraits : {all_zphot.shape}")


--- 3. PRÉPARATION DES DONNÉES GLOBALES ---
⚠️ ZSPEC introuvable dans un fichier. Remplissage avec des NaN.
⚠️ ZSPEC introuvable dans un fichier. Remplissage avec des NaN.
✅ Magnitudes extraites : (12524, 8)
✅ ZSPEC (Vérité Terrain / Test) extraits : (12524,)
✅ ZPHOT (Estimation / Train) extraits : (12524,)


In [38]:
# ==========================================
# NETTOYAGE DES BANDES VIDES ET CRÉATION DU DATAFRAME
# ==========================================

print("\n--- NETTOYAGE DES COLONNES MORTES (Variance nulle) ---")
# On identifie les colonnes de magnitudes qui sont pleines de 0 (pour éviter les NaN dans le VIF)
variances = np.var(all_mags, axis=0)
colonnes_valides_idx = np.where(variances > 0)[0]
colonnes_vides_idx = np.where(variances == 0)[0]

if len(colonnes_vides_idx) > 0:
    bandes_vides = [bands[i] for i in colonnes_vides_idx]
    print(f"⚠️ Alerte : Suppression des bandes vides ou constantes : {bandes_vides}")
    
    # On met à jour les matrices et les listes pour la suite du script
    all_mags = all_mags[:, colonnes_valides_idx]
    bands = [bands[i] for i in colonnes_valides_idx]

# Création du DataFrame finalisé avec les bandes valides
df_data = pd.DataFrame(all_mags, columns=bands)
for i in range(len(bands) - 1):
    col1, col2 = bands[i], bands[i+1]
    df_data[f"{col1}-{col2}"] = df_data[col1] - df_data[col2]


--- NETTOYAGE DES COLONNES MORTES (Variance nulle) ---


In [39]:
# ==========================================
# 4. EXTRACTION DES FEATURES (RÉSEAU SSL)
# ==========================================
print("\n--- INJECTION DES FEATURES DU RÉSEAU SSL ---")

# (N, 64, 64, 9) -> (N, 9, 64, 64)
cubes_transposed = np.transpose(all_cubes, (0, 3, 1, 2))
tensor_cubes = torch.FloatTensor(cubes_transposed)
tensor_cubes = torch.nan_to_num(tensor_cubes, nan=0.0, posinf=0.0, neginf=0.0)

dataset = TensorDataset(tensor_cubes)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

class ResNetAstroExtractor(nn.Module):
    def __init__(self, architecture='resnet18'):
        super().__init__()
        if architecture == 'resnet18':
            self.backbone = models.resnet18(weights=None)
        elif architecture == 'resnet50':
            self.backbone = models.resnet50(weights=None)
        else:
            raise ValueError("Architecture non supportée.")

        self.backbone.conv1 = nn.Conv2d(9, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        return self.backbone(x)

model_ssl = ResNetAstroExtractor(architecture='resnet18')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ssl = model_ssl.to(device)
model_ssl.eval()

print(f"Extraction des features sur {device} en cours...")
features_list = []

with torch.no_grad():
    for batch in dataloader:
        inputs = batch[0].to(device)
        outputs = model_ssl(inputs)
        features_list.append(outputs.cpu().numpy())

all_nn_features = np.vstack(features_list)
print(f"✅ Features réseau extraites avec succès ! Shape finale : {all_nn_features.shape}")


--- INJECTION DES FEATURES DU RÉSEAU SSL ---
Extraction des features sur cuda en cours...
✅ Features réseau extraites avec succès ! Shape finale : (12524, 512)


In [40]:
# ==========================================
# 5. CALCUL DU VIF (Redondance linéaire)
# ==========================================
print("\n--- CALCUL DU VIF (Test de colinéarité) ---")
def compute_vif(df):
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna()
    df_with_const = df_clean.copy()
    df_with_const['const'] = 1.0
    
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df_with_const.columns
    vif_data["VIF"] = [variance_inflation_factor(df_with_const.values, i) for i in range(df_with_const.shape[1])]
    
    return vif_data[vif_data["Feature"] != "const"]

try:
    vif_results = compute_vif(df_data)
    print(vif_results)
except Exception as e:
    print(f"Erreur lors du calcul du VIF : {e}")


--- CALCUL DU VIF (Test de colinéarité) ---


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


   Feature  VIF
0        u  inf
1        g  inf
2        r  inf
3        i  inf
4        z  inf
5        y  inf
6        J  inf
7        H  inf
8      u-g  inf
9      g-r  inf
10     r-i  inf
11     i-z  inf
12     z-y  inf
13     y-J  inf
14     J-H  inf


In [41]:
# ==========================================
# 6. AUTO-ENCODEUR (Redondance Non-Linéaire)
# ==========================================

print("\n--- AUTO-ENCODEUR : MAGNITUDES -> FEATURES RÉSEAU ---")
class RedundancyAE(nn.Module):
    def __init__(self, in_features, target_features):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features, 32), nn.ReLU(),
            nn.Linear(32, 16), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32), nn.ReLU(),
            nn.Linear(32, target_features)
        )
        
    def forward(self, x):
        return self.decoder(self.encoder(x))

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_tensor = torch.FloatTensor(scaler_x.fit_transform(all_mags))
Y_tensor = torch.FloatTensor(scaler_y.fit_transform(all_nn_features))

model_ae = RedundancyAE(X_tensor.shape[1], Y_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model_ae.parameters(), lr=0.01)

for epoch in range(50):
    optimizer.zero_grad()
    predictions = model_ae(X_tensor)
    loss = criterion(predictions, Y_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], MSE Loss : {loss.item():.4f}")


--- AUTO-ENCODEUR : MAGNITUDES -> FEATURES RÉSEAU ---
Epoch [10/50], MSE Loss : 0.9871
Epoch [20/50], MSE Loss : 0.9798
Epoch [30/50], MSE Loss : 0.9696
Epoch [40/50], MSE Loss : 0.9354
Epoch [50/50], MSE Loss : 0.8711


In [42]:
# ==========================================
# 7. RÉGRESSION (Train: ZPHOT / Test: ZSPEC)
# ==========================================

print("\n--- TESTS DE RÉGRESSION (Train: ZPHOT -> Test: ZSPEC) ---")

# 1. CRÉATION DES MASQUES LOGIQUES
# On garde pour le TEST uniquement les galaxies ayant un ZSPEC valide
mask_test_zspec = ~np.isnan(all_zspec) & (all_zspec > 0)

# On garde pour l'ENTRAÎNEMENT le reste des galaxies, si elles ont un ZPHOT valide
mask_train_zphot = (~mask_test_zspec) & (~np.isnan(all_zphot)) & (all_zphot > 0)

# 2. SÉPARATION DES DONNÉES D'ENTRAÎNEMENT (Les ~12000 ZPHOT)
X_train_mags = all_mags[mask_train_zphot]
X_train_nn = all_nn_features[mask_train_zphot]
X_train_combined = np.hstack((X_train_mags, X_train_nn))
y_train_zphot = all_zphot[mask_train_zphot]

# 3. SÉPARATION DES DONNÉES DE TEST (Les ~27 ZSPEC)
X_test_mags = all_mags[mask_test_zspec]
X_test_nn = all_nn_features[mask_test_zspec]
X_test_combined = np.hstack((X_test_mags, X_test_nn))
y_test_zspec = all_zspec[mask_test_zspec]

print(f"Galaxies pour l'ENTRAÎNEMENT (sur ZPHOT) : {len(y_train_zphot)}")
print(f"Galaxies pour le TEST (sur ZSPEC)        : {len(y_test_zspec)}")

# 4. ENTRAÎNEMENT
rf_mags = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
rf_combined = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)

print("\nEntraînement Modèle A (Magnitudes seules) sur ZPHOT...")
rf_mags.fit(X_train_mags, y_train_zphot)

print("Entraînement Modèle B (Magnitudes + Features NN) sur ZPHOT...")
rf_combined.fit(X_train_combined, y_train_zphot)

# 5. ÉVALUATION
score_mags = r2_score(y_test_zspec, rf_mags.predict(X_test_mags))
score_combined = r2_score(y_test_zspec, rf_combined.predict(X_test_combined))

print("\n--- RÉSULTATS (Prédictions évaluées contre ZSPEC) ---")
print(f"Magnitudes seules        : {score_mags:.4f}")
print(f"Magnitudes + Features NN : {score_combined:.4f}")

if len(y_test_zspec) < 50:
    print("⚠️ Note : Tester sur un si petit nombre de ZSPEC peut rendre le score R2 très volatile.")


--- TESTS DE RÉGRESSION (Train: ZPHOT -> Test: ZSPEC) ---
Galaxies pour l'ENTRAÎNEMENT (sur ZPHOT) : 12497
Galaxies pour le TEST (sur ZSPEC)        : 27

Entraînement Modèle A (Magnitudes seules) sur ZPHOT...
Entraînement Modèle B (Magnitudes + Features NN) sur ZPHOT...

--- RÉSULTATS (Prédictions évaluées contre ZSPEC) ---
Magnitudes seules        : 0.4174
Magnitudes + Features NN : 0.4054
⚠️ Note : Tester sur un si petit nombre de ZSPEC peut rendre le score R2 très volatile.


In [43]:
# ==========================================
# 8. TEST DU 'RÉSULTAT PARFAIT' (Biais de Magnitude)
# ==========================================

print("\n--- TEST DU 'RÉSULTAT PARFAIT' (Biais de Magnitude) ---")
colonnes_mags = bands
colonnes_couleurs = [c for c in df_data.columns if "-" in c]

X_mags_only = df_data[colonnes_mags].values
X_colors_only = df_data[colonnes_couleurs].values

modele_test_mags = Ridge()
modele_test_colors = Ridge()

modele_test_mags.fit(X_mags_only, all_nn_features)
pred_from_mags = modele_test_mags.predict(X_mags_only)
r2_mags_to_ssl = r2_score(all_nn_features, pred_from_mags)

modele_test_colors.fit(X_colors_only, all_nn_features)
pred_from_colors = modele_test_colors.predict(X_colors_only)
r2_colors_to_ssl = r2_score(all_nn_features, pred_from_colors)

print(f"-> R2 Score (Magnitudes Absolues -> SSL Features) : {r2_mags_to_ssl:.4f}")
print(f"-> R2 Score (Couleurs/Différences -> SSL Features) : {r2_colors_to_ssl:.4f}")

if r2_colors_to_ssl > r2_mags_to_ssl + 0.1:
    print("\n✅ RÉSULTAT PARFAIT ATTEINT !")
    print("Les features sont corrélées aux différences mais ont perdu l'information de la magnitude absolue.")
elif r2_mags_to_ssl > 0.8:
    print("\n❌ BIAIS PRÉSENT : Le réseau encode la magnitude absolue dans ses features.")
else:
    print("\n⚠️ RÉSULTAT MITIGÉ : Le réseau encode des informations complexes non réductibles aux magnitudes ou couleurs globales.")


--- TEST DU 'RÉSULTAT PARFAIT' (Biais de Magnitude) ---
-> R2 Score (Magnitudes Absolues -> SSL Features) : 0.0185
-> R2 Score (Couleurs/Différences -> SSL Features) : 0.0114

⚠️ RÉSULTAT MITIGÉ : Le réseau encode des informations complexes non réductibles aux magnitudes ou couleurs globales.


# Sans valeurs aberrantes :

In [44]:
# ==========================================
# 3.2. CONCATÉNATION ET PRÉPARATION DES DONNÉES
# ==========================================

print("\n--- 3.2. PRÉPARATION DES DONNÉES GLOBALES ---")

bands = ["u", "g", "r", "i", "z", "y", "J", "H"]

mags_list = []
zspec_list = []
zphot_list = []
cube_list = []

for d in data_list:
    info_array = d['info']
    
    mags = np.column_stack([info_array[b] for b in bands])
    mags_list.append(mags)
    zphot_list.append(info_array['ZPHOT'])
    cube_list.append(d['cube'])
    
    # Extraction de ZSPEC avec protection NaN
    try:
        zspec_list.append(info_array['ZSPEC'])
    except ValueError:
        print(f"⚠️ ZSPEC introuvable dans un fichier. Remplissage avec des NaN.")
        zspec_list.append(np.full(info_array.shape[0], np.nan))

all_mags = np.vstack(mags_list)
all_zphot = np.hstack(zphot_list)
all_zspec = np.hstack(zspec_list)
all_cubes = np.vstack(cube_list)

print(f"✅ Magnitudes extraites : {all_mags.shape}")
print(f"✅ ZSPEC (Vérité Terrain / Test) extraits : {all_zspec.shape}")
print(f"✅ ZPHOT (Estimation / Train) extraits : {all_zphot.shape}")


--- 3.2. PRÉPARATION DES DONNÉES GLOBALES ---
⚠️ ZSPEC introuvable dans un fichier. Remplissage avec des NaN.
⚠️ ZSPEC introuvable dans un fichier. Remplissage avec des NaN.
✅ Magnitudes extraites : (12524, 8)
✅ ZSPEC (Vérité Terrain / Test) extraits : (12524,)
✅ ZPHOT (Estimation / Train) extraits : (12524,)


In [45]:
print("\n--- REMPLACEMENT DES VALEURS ABERRANTES PAR 0 ---")
mask_physique_mags = (all_mags < 0) | (all_mags > 40)
all_mags[mask_physique_mags] = 0

z_scores_mags = np.abs(stats.zscore(all_mags, nan_policy='omit'))
all_mags[z_scores_mags > 3] = 0

# Filtre avec protection pour ignorer les NaN lors de la comparaison
with np.errstate(invalid='ignore'):
    mask_physique_zspec = (all_zspec < 0) | (all_zspec > 6)
    mask_physique_zphot = (all_zphot < 0) | (all_zphot > 6)

# On ne met à 0 que les valeurs aberrantes. Les NaN restent des NaN.
all_zspec[mask_physique_zspec] = 0
all_zphot[mask_physique_zphot] = 0

print("✅ Valeurs aberrantes remplacées par 0 (Les ZSPEC manquants sont préservés en NaN).")


--- REMPLACEMENT DES VALEURS ABERRANTES PAR 0 ---
✅ Valeurs aberrantes remplacées par 0 (Les ZSPEC manquants sont préservés en NaN).


In [46]:
# ==========================================
# 3.3. NETTOYAGE DES BANDES VIDES ET CRÉATION DU DATAFRAME
# ==========================================
print("\n--- NETTOYAGE DES COLONNES MORTES (Variance nulle) ---")
# On identifie les colonnes de magnitudes qui sont pleines de 0 (pour éviter les NaN dans le VIF)
variances = np.var(all_mags, axis=0)
colonnes_valides_idx = np.where(variances > 0)[0]
colonnes_vides_idx = np.where(variances == 0)[0]

if len(colonnes_vides_idx) > 0:
    bandes_vides = [bands[i] for i in colonnes_vides_idx]
    print(f"⚠️ Alerte : Suppression des bandes vides ou constantes : {bandes_vides}")
    
    # On met à jour les matrices et les listes pour la suite du script
    all_mags = all_mags[:, colonnes_valides_idx]
    bands = [bands[i] for i in colonnes_valides_idx]

# Création du DataFrame finalisé avec les bandes valides
df_data = pd.DataFrame(all_mags, columns=bands)
for i in range(len(bands) - 1):
    col1, col2 = bands[i], bands[i+1]
    df_data[f"{col1}-{col2}"] = df_data[col1] - df_data[col2]


--- NETTOYAGE DES COLONNES MORTES (Variance nulle) ---
⚠️ Alerte : Suppression des bandes vides ou constantes : ['J', 'H']


In [47]:
# ==========================================
# 4. EXTRACTION DES FEATURES (RÉSEAU SSL)
# ==========================================

print("\n--- INJECTION DES FEATURES DU RÉSEAU SSL ---")

# (N, 64, 64, 9) -> (N, 9, 64, 64)
cubes_transposed = np.transpose(all_cubes, (0, 3, 1, 2))
tensor_cubes = torch.FloatTensor(cubes_transposed)
tensor_cubes = torch.nan_to_num(tensor_cubes, nan=0.0, posinf=0.0, neginf=0.0)

dataset = TensorDataset(tensor_cubes)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

class ResNetAstroExtractor(nn.Module):
    def __init__(self, architecture='resnet18'):
        super().__init__()
        if architecture == 'resnet18':
            self.backbone = models.resnet18(weights=None)
        elif architecture == 'resnet50':
            self.backbone = models.resnet50(weights=None)
        else:
            raise ValueError("Architecture non supportée.")

        self.backbone.conv1 = nn.Conv2d(9, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.backbone.fc = nn.Identity()

    def forward(self, x):
        return self.backbone(x)

model_ssl = ResNetAstroExtractor(architecture='resnet18')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ssl = model_ssl.to(device)
model_ssl.eval()

print(f"Extraction des features sur {device} en cours...")
features_list = []

with torch.no_grad():
    for batch in dataloader:
        inputs = batch[0].to(device)
        outputs = model_ssl(inputs)
        features_list.append(outputs.cpu().numpy())

all_nn_features = np.vstack(features_list)
print(f"✅ Features réseau extraites avec succès ! Shape finale : {all_nn_features.shape}")


--- INJECTION DES FEATURES DU RÉSEAU SSL ---
Extraction des features sur cuda en cours...
✅ Features réseau extraites avec succès ! Shape finale : (12524, 512)


In [48]:
# ==========================================
# 5. CALCUL DU VIF (Redondance linéaire)
# ==========================================
print("\n--- CALCUL DU VIF (Test de colinéarité) ---")
def compute_vif(df):
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna()
    df_with_const = df_clean.copy()
    df_with_const['const'] = 1.0
    
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df_with_const.columns
    vif_data["VIF"] = [variance_inflation_factor(df_with_const.values, i) for i in range(df_with_const.shape[1])]
    
    return vif_data[vif_data["Feature"] != "const"]

try:
    vif_results = compute_vif(df_data)
    print(vif_results)
except Exception as e:
    print(f"Erreur lors du calcul du VIF : {e}")


--- CALCUL DU VIF (Test de colinéarité) ---


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


   Feature  VIF
0        u  inf
1        g  inf
2        r  inf
3        i  inf
4        z  inf
5        y  inf
6      u-g  inf
7      g-r  inf
8      r-i  inf
9      i-z  inf
10     z-y  inf


In [49]:
# ==========================================
# 6. AUTO-ENCODEUR (Redondance Non-Linéaire)
# ==========================================

print("\n--- AUTO-ENCODEUR : MAGNITUDES -> FEATURES RÉSEAU ---")
class RedundancyAE(nn.Module):
    def __init__(self, in_features, target_features):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features, 32), nn.ReLU(),
            nn.Linear(32, 16), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32), nn.ReLU(),
            nn.Linear(32, target_features)
        )
        
    def forward(self, x):
        return self.decoder(self.encoder(x))

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_tensor = torch.FloatTensor(scaler_x.fit_transform(all_mags))
Y_tensor = torch.FloatTensor(scaler_y.fit_transform(all_nn_features))

model_ae = RedundancyAE(X_tensor.shape[1], Y_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model_ae.parameters(), lr=0.01)

for epoch in range(50):
    optimizer.zero_grad()
    predictions = model_ae(X_tensor)
    loss = criterion(predictions, Y_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], MSE Loss : {loss.item():.4f}")


--- AUTO-ENCODEUR : MAGNITUDES -> FEATURES RÉSEAU ---
Epoch [10/50], MSE Loss : 0.9999
Epoch [20/50], MSE Loss : 0.9939
Epoch [30/50], MSE Loss : 0.9754
Epoch [40/50], MSE Loss : 0.9162
Epoch [50/50], MSE Loss : 0.7729


In [50]:
# ==========================================
# 7. RÉGRESSION (Train: ZPHOT / Test: ZSPEC)
# ==========================================

print("\n--- TESTS DE RÉGRESSION (Train: ZPHOT -> Test: ZSPEC) ---")

# 1. CRÉATION DES MASQUES LOGIQUES
# On garde pour le TEST uniquement les galaxies ayant un ZSPEC valide
mask_test_zspec = ~np.isnan(all_zspec) & (all_zspec > 0)

# On garde pour l'ENTRAÎNEMENT le reste des galaxies, si elles ont un ZPHOT valide
mask_train_zphot = (~mask_test_zspec) & (~np.isnan(all_zphot)) & (all_zphot > 0)

# 2. SÉPARATION DES DONNÉES D'ENTRAÎNEMENT (Les ~12000 ZPHOT)
X_train_mags = all_mags[mask_train_zphot]
X_train_nn = all_nn_features[mask_train_zphot]
X_train_combined = np.hstack((X_train_mags, X_train_nn))
y_train_zphot = all_zphot[mask_train_zphot]

# 3. SÉPARATION DES DONNÉES DE TEST (Les ~27 ZSPEC)
X_test_mags = all_mags[mask_test_zspec]
X_test_nn = all_nn_features[mask_test_zspec]
X_test_combined = np.hstack((X_test_mags, X_test_nn))
y_test_zspec = all_zspec[mask_test_zspec]

print(f"Galaxies pour l'ENTRAÎNEMENT (sur ZPHOT) : {len(y_train_zphot)}")
print(f"Galaxies pour le TEST (sur ZSPEC)        : {len(y_test_zspec)}")

# 4. ENTRAÎNEMENT
rf_mags = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
rf_combined = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)

print("\nEntraînement Modèle A (Magnitudes seules) sur ZPHOT...")
rf_mags.fit(X_train_mags, y_train_zphot)

print("Entraînement Modèle B (Magnitudes + Features NN) sur ZPHOT...")
rf_combined.fit(X_train_combined, y_train_zphot)

# 5. ÉVALUATION
score_mags = r2_score(y_test_zspec, rf_mags.predict(X_test_mags))
score_combined = r2_score(y_test_zspec, rf_combined.predict(X_test_combined))

print("\n--- RÉSULTATS (Prédictions évaluées contre ZSPEC) ---")
print(f"Magnitudes seules        : {score_mags:.4f}")
print(f"Magnitudes + Features NN : {score_combined:.4f}")

if len(y_test_zspec) < 50:
    print("⚠️ Note : Tester sur un si petit nombre de ZSPEC peut rendre le score R2 très volatile.")


--- TESTS DE RÉGRESSION (Train: ZPHOT -> Test: ZSPEC) ---
Galaxies pour l'ENTRAÎNEMENT (sur ZPHOT) : 12497
Galaxies pour le TEST (sur ZSPEC)        : 27

Entraînement Modèle A (Magnitudes seules) sur ZPHOT...
Entraînement Modèle B (Magnitudes + Features NN) sur ZPHOT...

--- RÉSULTATS (Prédictions évaluées contre ZSPEC) ---
Magnitudes seules        : 0.4082
Magnitudes + Features NN : 0.4113
⚠️ Note : Tester sur un si petit nombre de ZSPEC peut rendre le score R2 très volatile.


In [51]:
# ==========================================
# 8.2. TEST DU 'RÉSULTAT PARFAIT' (Biais de Magnitude)
# ==========================================

print("\n--- TEST DU 'RÉSULTAT PARFAIT' (Biais de Magnitude) ---")
colonnes_mags = bands
colonnes_couleurs = [c for c in df_data.columns if "-" in c]

X_mags_only = df_data[colonnes_mags].values
X_colors_only = df_data[colonnes_couleurs].values

modele_test_mags = Ridge()
modele_test_colors = Ridge()

modele_test_mags.fit(X_mags_only, all_nn_features)
pred_from_mags = modele_test_mags.predict(X_mags_only)
r2_mags_to_ssl = r2_score(all_nn_features, pred_from_mags)

modele_test_colors.fit(X_colors_only, all_nn_features)
pred_from_colors = modele_test_colors.predict(X_colors_only)
r2_colors_to_ssl = r2_score(all_nn_features, pred_from_colors)

print(f"-> R2 Score (Magnitudes Absolues -> SSL Features) : {r2_mags_to_ssl:.4f}")
print(f"-> R2 Score (Couleurs/Différences -> SSL Features) : {r2_colors_to_ssl:.4f}")

if r2_colors_to_ssl > r2_mags_to_ssl + 0.1:
    print("\n✅ RÉSULTAT PARFAIT ATTEINT !")
    print("Les features sont corrélées aux différences mais ont perdu l'information de la magnitude absolue.")
elif r2_mags_to_ssl > 0.8:
    print("\n❌ BIAIS PRÉSENT : Le réseau encode la magnitude absolue dans ses features.")
else:
    print("\n⚠️ RÉSULTAT MITIGÉ : Le réseau encode des informations complexes non réductibles aux magnitudes ou couleurs globales.")


--- TEST DU 'RÉSULTAT PARFAIT' (Biais de Magnitude) ---
-> R2 Score (Magnitudes Absolues -> SSL Features) : 0.0030
-> R2 Score (Couleurs/Différences -> SSL Features) : 0.0001

⚠️ RÉSULTAT MITIGÉ : Le réseau encode des informations complexes non réductibles aux magnitudes ou couleurs globales.
